# InSAR coherence timeseries

This notebook demonstrates how to generate InSAR coherence time series for all Sentinel-1 bursts defined in a given GeoJSON file (e.g., `s1_bursts_glacierSouthTyrol.geojson`). The GeoJSON file is a filtered subset of the Burst ID map available from the SAR-MPC products (https://sar-mpc.eu/sar-mpc-products/), containing information about subswath name and burst ID for each burst.

In this example, we generate a time series of 12 days InSAR coherence for the summer months of 2023, using all Sentinel-1 bursts that cover the glaciers of South Tyrol.

In [1]:
import openeo
import geopandas as gpd
import os
import numpy as np
import urllib
import json
from datetime import datetime
import shapely
from shapely import geometry
from shapely.ops import unary_union
import geopandas as gpd

## Input setting and preparation

In [2]:
start_date, end_date = ["2023-07-01", "2023-09-30"]

output_folder = './coh_timseries/'

bursts_filename = 's1_bursts_glacierSouthTyrol.geojson'
bursts = gpd.read_file(bursts_filename)

In [3]:
url = "https://openeo.dataspace.copernicus.eu"
connection = openeo.connect(url).authenticate_oidc()

Authenticated using refresh token.


In [5]:
def create_InSARpairs(burst_id, subswath_id, temporal_baseline):

    https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
        f"ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T23:59:59.000Z and "
        f"PolarisationChannels eq 'VV' and "
        f"BurstId eq {burst_id} and SwathIdentifier eq '{subswath_id}'"
    ) + "&$top=1000"
    
    with urllib.request.urlopen(https_request) as response:
        content = response.read().decode()
        s1_bursts = json.loads(content)

    dates = [datetime.strptime(b['BeginningDateTime'][:10], "%Y-%m-%d") for b in s1_bursts['value']]
    dates.sort()
    InSARpairs = []
    for date_ref in dates:
        for date_sec in dates:
            if (date_ref - date_sec).days == -temporal_baseline:
                InSARpairs.append([
                    datetime.strftime(date_ref, "%Y-%m-%d"),
                    datetime.strftime(date_sec, "%Y-%m-%d")
                ])

    return InSARpairs

## Create and start all the jobs (one job for each burst)

SAR coherence parallel processing with InSAR pair list as input

In [7]:
for idx, b in bursts.iterrows():
    jobs_list = list(connection.list_jobs())
    job_title = f"sar_coherence_T{b['relative_orbit_number']}_{b['subswath_name']}_{b['burst_id']}"
    job_completed_or_running = False

    for j in jobs_list:
        if "title" in j and j["title"]==job_title:
                # Check if existing job was completed, if not remove it and recreate
                if j["status"]=="error":
                    connection.job(j["id"]).delete()
                else:
                    job_completed_or_running = True
                break

    if not job_completed_or_running:
        datacube = connection.datacube_from_process(
            process_id="sar_coherence_parallel",
            InSAR_pairs=create_InSARpairs(int(b['burst_id']), b['subswath_name'], 12),
            burst_id=int(b['burst_id']),
            polarization="vv",
            sub_swath=b['subswath_name'],
            coherence_window_rg=21,
            coherence_window_az=5,
        )
        datacube = datacube.save_result(format='GTiff')
        job = datacube.create_job(
            title=job_title,
        )
        job.start()

### Check jobs status

Get the list of jobs and check if all of them are completed. Once they are all finished, combine the coherence of the selected bursts for each orbit track.

In [6]:
jobs_list = connection.list_jobs()
jobs_list

[{'created': '2026-01-22T16:12:20Z',
  'id': 'j-2601221612204ba09b2a8783673eb637',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T168',
  'updated': '2026-01-22T16:15:15Z'},
 {'created': '2026-01-22T16:11:30Z',
  'id': 'j-26012216113044638df4f3f809ddc35b',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117',
  'updated': '2026-01-22T16:15:17Z'},
 {'created': '2026-01-22T16:10:21Z',
  'id': 'j-2601221610214ca8b9adcd328061dda4',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T95',
  'updated': '2026-01-22T16:13:12Z'},
 {'created': '2026-01-22T16:09:37Z',
  'id': 'j-260122160937464b923308b9f2cadd92',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T44',
  'updated': '2026-01-22T16:12:12Z'},
 {'created': '2026-01-21T08:08:10Z',
  'id': 'j-2601210808104106bc36e4b4a21da81d',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW3_249436',
  'updated': '2026-01-21T09:13:21Z'},
 {'created': '2026-01-20T13:37:07Z',
  'id': 'j-2601201337074c98b1dfa15fa6274c9b',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T168_IW2_359500',
  'updated': '2026-01-20T16:57:28Z'},
 {'created': '2026-01-20T13:36:51Z',
  'id': 'j-260120133651480682e3deef6a5a0b3a',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T168_IW2_359499',
  'updated': '2026-01-20T16:50:20Z'},
 {'created': '2026-01-20T13:36:36Z',
  'id': 'j-26012013363645f88fa7998cf88f14e7',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T168_IW1_359499',
  'updated': '2026-01-20T16:46:36Z'},
 {'created': '2026-01-20T13:36:21Z',
  'id': 'j-26012013362149008633fb30b0690a82',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T168_IW1_359498',
  'updated': '2026-01-20T16:06:42Z'},
 {'created': '2026-01-20T13:35:50Z',
  'id': 'j-260120133550490698d08164be4afc60',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW2_249436',
  'updated': '2026-01-20T16:00:04Z'},
 {'created': '2026-01-20T13:35:34Z',
  'id': 'j-2601201335344c139faab82f5c7db0a6',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW1_249436',
  'updated': '2026-01-20T15:44:32Z'},
 {'created': '2026-01-20T13:35:18Z',
  'id': 'j-260120133518455787fa984464c00f8b',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW3_249435',
  'updated': '2026-01-20T15:53:25Z'},
 {'created': '2026-01-20T13:35:04Z',
  'id': 'j-2601201335044a698aa2797ea4c5104b',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW2_249435',
  'updated': '2026-01-20T15:41:49Z'},
 {'created': '2026-01-20T13:34:46Z',
  'id': 'j-260120133446429e82be550370cdc4d3',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW1_249435',
  'updated': '2026-01-20T15:33:27Z'},
 {'created': '2026-01-20T13:34:31Z',
  'id': 'j-260120133431422797b3ac6137429c36',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW2_249434',
  'updated': '2026-01-20T15:38:33Z'},
 {'created': '2026-01-20T13:34:17Z',
  'id': 'j-26012013341749fc99dbf98c882ef6c8',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T117_IW1_249434',
  'updated': '2026-01-20T15:26:49Z'},
 {'created': '2026-01-20T13:34:02Z',
  'id': 'j-26012013340241d6afb71d8a831406f5',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T95_IW3_202702',
  'updated': '2026-01-20T14:35:37Z'},
 {'created': '2026-01-20T13:33:46Z',
  'id': 'j-26012013334642c4b1af0b864733ba4c',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T95_IW3_202701',
  'updated': '2026-01-20T14:35:39Z'},
 {'created': '2026-01-20T13:33:31Z',
  'id': 'j-2601201333314124b64158d874ad92f3',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_T95_IW2_202701',
  'updated': '2026-01-20T14:39:38Z'},
 {'created': '2026-01-20T13:33:16Z',
  'id': 'j-2601201333164b759ccff3ff8c2c6a6d',
  'progress': 

In [15]:
def _to_2d(x, y, z):
    return tuple(filter(None, [x, y]))

jobs_list = list(connection.list_jobs())
orbits = np.unique(bursts["relative_orbit_number"])

for orbit in orbits:
    bursts_orbit = bursts[bursts["relative_orbit_number"]==orbit]
    bursts_id = np.unique(bursts_orbit["burst_id"])
    bboxes = []
    all_orbit_jobs_done = []
    cube = None
    for burst_id in bursts_id:
        for subswath in bursts_orbit[bursts_orbit["burst_id"]==burst_id]["subswath_name"]:
            geom = bursts_orbit[bursts_orbit["burst_id"]==burst_id][bursts_orbit["subswath_name"]==subswath].geometry
            burst_polygon = shapely.ops.transform(_to_2d, geom.item().geoms[0])
            bboxes.append(burst_polygon)

            job_title = f"sar_coherence_T{orbit}_{subswath}_{burst_id}"
            #Check if all jobs for the orbit are completed, so that we can merge them
            for j in jobs_list:
                if "title" in j and j["title"]==job_title:
                        if j["status"]=="finished":
                            all_orbit_jobs_done.append(True)
                            if cube is None:
                                cube = connection.load_stac_from_job(j["id"])
                            else:
                                cube = cube.merge_cubes(connection.load_stac_from_job(j["id"]),overlap_resolver="max")
                        else:
                            all_orbit_jobs_done.append(False)
                        break
    if not np.all(all_orbit_jobs_done):
        print("Not all jobs for orbit {} completed, skipping this orbit.\nCheck the openEO jobs, since there might jobs which are still running.\nRun again the previous code block if necessary, which would delete and relaunch failed jobs if there's any.")
        continue
    job_completed_or_running = False
    for j in jobs_list:
        if "title" in j and j["title"]==f"sar_coherence_T{orbit}":
            # Check if existing job was completed, if not remove it and recreate
            if j["status"]=="error":
                connection.job(j["id"]).delete()
            else:
                job_completed_or_running = True
            break

    if not job_completed_or_running:
        boundary = gpd.GeoSeries(unary_union(bboxes))
        cube = cube.filter_bbox(list(boundary.bounds.values[0]))
        cube_tiff = cube.save_result(format="GTiff")
        job = cube_tiff.create_job(title=f"sar_coherence_T{orbit}",job_options={"python-memory": "4000m"})
        job.start_job()


/home/mclaus@eurac.edu/micromamba/envs/eopf-zarr/lib/python3.11/site-packages/geopandas/geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
bands_from_stac_collection: consulting items for band metadata
/home/mclaus@eurac.edu/micromamba/envs/eopf-zarr/lib/python3.11/site-packages/geopandas/geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
bands_from_stac_collection: consulting items for band metadata
/home/mclaus@eurac.edu/micromamba/envs/eopf-zarr/lib/python3.11/site-packages/geopandas/geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
bands_from_stac_collection: consulting items for band metadata
/home/mclaus@eurac.edu/micromamba/envs/eopf-zarr/lib/python3.11/site-packages/geopandas/geodataframe.py:1896: UserWarning: Boolean Series key wil

Once that all the merged coherence jobs are aompleted, we can download the resulting files:

In [16]:
jobs_list = list(connection.list_jobs())
orbits = np.unique(bursts["relative_orbit_number"])

for orbit in orbits:
    for j in jobs_list:
        if "title" in j and j["title"]==f"sar_coherence_T{orbit}":
            # Check if existing job was completed, if not remove it and recreate
            if j["status"]=="finished":
                connection.job(j["id"]).get_results().download_files(
                    os.path.join(output_folder, f"sar_coherence_T{orbit}")
                    )
            break